In [6]:
%load_ext autoreload
%autoreload 2

import math
import sys

import numpy as np

sys.path.append('../')
sys.path.append('../../')
from run_utils import get_env_dataset, run_env_experiment
from run_utils import ModelTuner
from reclab.environments import LatentFactorBehavior
from env_defaults import TOPICS_STATIC, get_len_trial, LATENT_STATIC
from reclab.recommenders import EASE


In [7]:
# ====Step 4====
# S3 storage parameters
bucket_name = None # 'recsys-eval'
data_dir = 'master'
overwrite = True

# Experiment setup.
n_trials = 10
trial_seeds = [i for i in range(n_trials)]
len_trial = get_len_trial(LATENT_STATIC)

# Environment setup
environment_name = LATENT_STATIC['name']
env = LatentFactorBehavior(**LATENT_STATIC['params'], **LATENT_STATIC['optional_params'])

# Recommender setup
recommender_name = 'EASE'
recommender_class = EASE


In [8]:
# ====Step 5====
starting_data = get_env_dataset(env)


In [36]:
# ====Step 6====
# Recommender tuning setup
n_fold = 5
default_params = {}
tuner = ModelTuner(starting_data,
                   default_params,
                   recommender_class,
                   n_fold=n_fold,
                   verbose=True,
                   bucket_name=bucket_name,
                   data_dir=data_dir,
                   environment_name=environment_name,
                   recommender_name=recommender_name,
                   overwrite=overwrite,
                  use_mse=False)



In [39]:
# Tune the hyperparameter.
# Start with a coarse grid.
lams = np.linspace(10, 9000, 10).tolist()
tuner.evaluate_grid(lam=lams)


Evaluating: {'lam': 10.0}
Fold 1/5, dcg=1159.2451969282708, ndcg=0.655543109171518
Fold 2/5, dcg=1175.6155485340212, ndcg=0.6637645013359991
Fold 3/5, dcg=1159.6585698484637, ndcg=0.6545976901945396
Fold 4/5, dcg=1172.4575945271142, ndcg=0.663094623078153
Fold 5/5, dcg=1156.7119298988707, ndcg=0.6550418370196125
Average Metric: 0.6584083521599645
Evaluating: {'lam': 1008.8888888888889}
Fold 1/5, dcg=1238.9410686240117, ndcg=0.7006104337185551
Fold 2/5, dcg=1249.9452229363771, ndcg=0.7057317918551405
Fold 3/5, dcg=1245.366586844665, ndcg=0.7029776801464089
Fold 4/5, 

KeyboardInterrupt: 

In [40]:
# Zeroing in on the best range.
lams = np.linspace(5000, 15000, 10).tolist()
tuner.evaluate_grid(lam=lams)

Evaluating: {'lam': 5000.0}
Fold 1/5, dcg=1318.3204524524656, ndcg=0.745498787120328
Fold 2/5, dcg=1325.2932206471487, ndcg=0.7482740380762987
Fold 3/5, dcg=1326.7800931942286, ndcg=0.7489335283526846
Fold 4/5, dcg=1333.1468068860163, ndcg=0.75397394630419
Fold 5/5, dcg=1305.5763207217142, ndcg=0.7393432101712893
Average Metric: 0.7472047020049581
Evaluating: {'lam': 6111.111111111111}
Fold 1/5, dcg=1331.8233183845196, ndcg=0.7531345407462906
Fold 2/5, dcg=1338.854279920175, ndcg=0.7559307501342267
Fold 3/5, dcg=1336.866553648885, ndcg=0.7546270780642341
Fold 4/5, dcg=1344.2043847056332, ndcg=0.7602276653561046
Fold 5/5, dcg=1313.3659825971997, ndcg=0.7437544678861863
Average Metric: 0.7535349004374085
Evaluating: {'lam': 7222.222222222223}
Fold 1/5, dcg=1343.2486766778034, ndcg=0.7595954818127916
Fold 2/5, dcg=1347.8892874947371, ndcg=0.7610320073477559
Fold 3/5, dcg=1348.2797954165792, ndcg=0.7610695619927069
Fold 4/5, dcg=1355.038842261427, ndcg=0.766355196605636
Fold 5/5, dcg=1325.

,lam,metric,average_metric
0,5000.000000,"[0.745498787120328, 0.7482740380762987, 0.7489...",0.747205
1,6111.111111,"[0.7531345407462906, 0.7559307501342267, 0.754...",0.753535
2,7222.222222,"[0.7595954818127916, 0.7610320073477559, 0.761...",0.759724
3,8333.333333,"[0.7647372020157827, 0.7656777771075933, 0.766...",0.764612
4,9444.444444,"[0.7690536612132566, 0.7708915285983514, 0.769...",0.768996
5,10555.555556,"[0.7729287484225068, 0.7751896180140095, 0.774...",0.772973
6,11666.666667,"[0.7758002910531748, 0.7779306079474518, 0.777...",0.776013
7,12777.777778,"[0.7781965197071199, 0.780117366293597, 0.7809...",0.778849
8,13888.888889,"[0.7802536463621063, 0.7827041995294448, 0.782...",0.781412
9,15000.000000,"[0.7819898923990856, 0.7849891505552608, 0.784...",0.783363


In [ ]:

# Set regularization to 5889.
lam = 5889


In [ ]:

# ====Step 7====
recommender = recommender_class(shrinkage=shrinkage, neighborhood_size=neighborhood_size)
for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite)
